# Analyzing River Thames Water Levels
Time series data is everywhere, from watching your stock portfolio to monitoring climate change, and even live-tracking as local cases of a virus become a global pandemic. In this project, you’ll work with a time series that tracks the tide levels of the Thames River. You’ll first load the data and inspect it data visually, and then perform calculations on the dataset to generate some summary statistics. You’ll end by reducing the time series to its component attributes and analyzing them. 

The original dataset is available from the British Oceanographic Data Center.

Here's a map of the locations of the tidal meters along the River Thames in London.

![](locations.png)

The provided datasets are in the `data` folder in this workspace. For this project, you will work with one of these files, `10-11_London_Bridge.txt`, which contains comma separated values for water levels in the Thames River at the London Bridge. After you've finished the project, you can use your same code to analyze data from the other files (at other spots in the UK where tidal data is collected) if you'd like. 

The TXT file contains data for three variables, described in the table below. 

| Variable Name | Description | Format |
| ------------- | ----------- | ------ |
| Date and time | Date and time of measurement to GMT. Note the tide gauge is accurate to one minute. | dd/mm/yyyy hh:mm:ss |
| Water level | High or low water level measured by tide meter. Tide gauges are accurate to 1 centimetre. | metres (Admiralty Chart Datum (CD), Ordnance Datum Newlyn (ODN or Trinity High Water (THW)) | 
| Flag | High water flag = 1, low water flag = 0 | Categorical (0 or 1) |



In [47]:
# We've imported your first Python package for you, along with a function you will need called IQR
import pandas as pd 
import numpy as np

def IQR(column): 
    """ Calculates the interquartile range (IQR) for a given DataFrame column using the quantile method """
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

# Import a dataframe using pandas
data = pd.read_csv("data/10-11_London_Bridge.txt")

# Check the dataframe for missing values and info
data.info()

# Rename columns to take out the space in the names
data.rename(columns={" water level (m ODN)":"Water level"," flag":"Flag"," HW=1 or LW=0":"High_vs_Low"}, inplace=True)

# Modify the type column Date and time to date.
data['Date and time'] = pd.to_datetime(data['Date and time'])

# Modify Water level (m ODN) to float
data["Water level"] = data["Water level"].astype(float)

# Obtain a Month and Year column to facilitate the calculations
data["Month"] = data["Date and time"].dt.month
data["Year"] = data["Date and time"].dt.year

# Create extra column in the df in order to classify in High and Low tie
def high_vs_low(x):
    if x > 0:
        return "High"
    else:
        return "Low"
# Apply the new formula to High_vs_Low column
data["High_vs_Low"] = data["Flag"].apply(high_vs_low)

# Filter the df into High and Low level
df_high = data[data["High_vs_Low"] == "High"]
df_low = data[data["High_vs_Low"] == "Low"]

# Obtain mean, median and IQR level for "High" and "Low" water leve
df_high_st = df_high["Water level"].agg(["mean","median", IQR])
df_low_st = df_low["Water level"].agg(["mean","median", IQR])

# Calculate days and very high tide days in each year for High tide days
high_days = df_high.groupby("Year")["Water level"].count()
very_high = df_high[df_high["Water level"] > df_high["Water level"].quantile(0.90)]
very_high_days = very_high.groupby("Year")["Water level"].count()
v_high_ratio = (very_high_days/high_days).reset_index()

# Calculate days and very low tide days in each year for Low tide days
low_days = df_low.groupby("Year")["Water level"].count()
very_low = df_low[df_low["Water level"] < df_low["Water level"].quantile(0.10)]
very_low_days = very_low.groupby("Year")["Water level"].count()
v_low_ratio = (very_low_days/low_days).reset_index()

# Dictionary with the solution
solution = {"high_statistics": df_high_st, "low_statistics": df_low_st, "very_high_ratio": v_high_ratio, "very_low_ratio": v_low_ratio}

print(solution)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115503 entries, 0 to 115502
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Date and time         115503 non-null  object 
 1    water level (m ODN)  115503 non-null  object 
 2    flag                 115503 non-null  int64  
 3    HW=1 or LW=0         0 non-null       float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.5+ MB
    Year  Water level
0   1911     0.004098
1   1912     0.032316
2   1913     0.082212
3   1914     0.055313
4   1915     0.045045
..   ...          ...
80  1991     0.096317
81  1992     0.103253
82  1993     0.145923
83  1994     0.150355
84  1995     0.170213

[85 rows x 2 columns]
